In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the CSV file to get the list of symbols
file_path = 'ind_niftytotalmarket_list.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Assuming the symbols are in the third column
symbols = list(df.iloc[:, 2].dropna().unique())  # Remove any NaN values and duplicates

In [ ]:
symbols_temp = []

for symbol in symbols:

    symbols_temp.append(symbol + '.NS')

In [ ]:
symbols_temp

In [ ]:
len(symbols_temp)

In [ ]:
symbols = symbols_temp

In [ ]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 1)

In [ ]:
# Function to fetch and return weekly data for a given symbol
def get_weekly_data(symbol):

    data = yf.download(symbol, start_date, end_date, interval="1d")
    return data

# Dictionary to hold data for all symbols
all_data = {}

# Loop through each symbol and fetch its weekly data
for symbol in symbols:
    df = get_weekly_data(symbol)
    all_data[symbol] = df

In [ ]:
close_columns = [df['Adj Close'] for df in all_data.values()]

final_stock_df = pd.DataFrame()
final_stock_df = pd.concat(close_columns, axis=1)
column_names = []
column_names += list(all_data.keys())
final_stock_df.columns = column_names

In [ ]:
final_stock_df.index = pd.to_datetime(final_stock_df.index)
final_stock_df.sort_index(inplace=True)

In [ ]:
final_stock_df

In [ ]:
final_stock_df.to_csv('Data.csv')

In [ ]:
final_stock_df = pd.read_csv('Data.csv', index_col='Date')

In [ ]:
final_stock_df

In [ ]:
final_symbols = list(final_stock_df.columns)

In [ ]:
len(final_symbols)

In [ ]:
one_week_returns_all_stocks = pd.DataFrame(index=final_stock_df.index)

for symbol in final_symbols:

    one_week_returns = []

    for i in range(len(final_stock_df)):
        if i+1 < len(final_stock_df):
            pct_change = ((final_stock_df[symbol].iloc[i+1] - final_stock_df[symbol].iloc[i])/final_stock_df[symbol].iloc[i])
        else:
            pct_change = 0
        one_week_returns.append(pct_change)

    one_week_returns_all_stocks[symbol] = one_week_returns

In [ ]:
one_week_returns_all_stocks

In [ ]:
covariance_matrix = one_week_returns_all_stocks.cov()

In [ ]:
covariance_matrix

In [ ]:
covariance_matrix.to_csv('covariance_matrix.csv')

In [ ]:
covariance_matrix = pd.read_csv('covariance_matrix.csv', index_col=0)

In [ ]:
covariance_matrix

In [ ]:
summary_stats = pd.DataFrame(columns=final_stock_df.columns)

In [ ]:
for symbol in final_stock_df.columns:
    x = one_week_returns_all_stocks[symbol].mean()
    summary_stats[symbol] = [x]

In [ ]:
summary_stats

In [ ]:
final_stock_df.to_csv('Final Stock Data.csv')